In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic App: Question & Answering from Document

In [2]:
from langchain_openai import OpenAI

In [3]:
llm = OpenAI()

### Load text file

In [4]:
from langchain.document_loaders import TextLoader

In [5]:
loader = TextLoader("data/be-good-and-how-not-to-die.txt")

In [6]:
document = loader.load()

### Document is loaded as Python list with metadata

In [7]:
print(type(document))

<class 'list'>


In [8]:
print(len(document))

1


In [9]:
print(document[0].metadata)

{'source': 'data/be-good-and-how-not-to-die.txt'}


In [10]:
print(f"You have {len(document)} document.")

You have 1 document.


In [11]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 27419 characters


### Split document in small chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [14]:
document_chunks = text_splitter.split_documents(document)

In [15]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 12 chunks.


### Convert text chunks in numeric vectors (called "embeddings")

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [17]:
embeddings = OpenAIEmbeddings()

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Load embeddings to vector database

In [18]:
from langchain.vectorstores import FAISS

In [19]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

### Create Retrieval Question & Answering Chain

In [20]:
from langchain.chains import RetrievalQA

In [21]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

### Now we have Question & Answering App

In [22]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [23]:
QA_chain.run(question)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nThis article is about the importance of perseverance and not giving up in startup companies. The author shares his experiences and observations from working with startups and emphasizes the need to keep pushing through difficult times in order to achieve success. He also discusses the role of motivation and the fear of failure in driving founders to succeed.'

## New way: with LCEL

In [24]:
from langchain.prompts import PromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [25]:
retriever = stored_embeddings.as_retriever()

In [26]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [27]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
chain.invoke("What is this article about? Describe it in less than 100 words.")

'\nThis article discusses the importance of creating something that people want in order to have a successful startup. It also emphasizes the idea of not worrying too much about the business model at first and instead focusing on building a great product. The article also touches on the concept of failure and how it can be a motivating factor for founders to succeed. Additionally, it mentions the high likelihood of facing challenges and obstacles in a startup, but encourages readers to not give up and to keep pushing through. Overall, the article highlights the key elements and mindset needed for a startup to thrive.'